# LLM'i otomatik ve çok yönlü değerlendirmek için bir yargıç gibi 🧑‍⚖️ kullanmak
_Yazan: [Aymeric Roucher](https://huggingface.co/m-ric)_

Büyük dil modellerinin (LLM'lerin) değerlendirilmesi genellikle zorlu bir iştir; geniş yetenekleri göz önüne alındığında, onlara verilen görevler çoğu zaman kapsamlı ve kesin sınırlarla belirlenmemiş gereksinimlere göre değerlendirilmelidir. Örneğin, bir asistanın bir soruya verdiği yanıt şu şekilde olabilir:

Örneğin, bir ajanın bir soruya verdiği yanıt şu şekilde olabilir:
- Bağlama dayalı olmaması
- Tekrarlayıcı olması (tekrar, tekrar, tekrarlayan)
- Dil bilgisi hataları içermesi
- Gereğinden fazla uzun olması ve kelime fazlalığı içermesi (söylemin veya yazılı içeriğin gereksiz derecede ayrıntılı ve uzatılmış bir hale gelmesi)
- Tutarsız olması
- ...

Kriterlerin listesi uzayıp gider. Üstelik sınırlı bir listeye sahip olsak bile bunların her birini ölçmek zordur: "çıktıları (output'ları) değerlendirmek için kural tabanlı bir program geliştirmek fazlasıyla zordur. Çıktılar ve referans yanıtlar arasındaki benzerliğe dayanan geleneksel değerlendirme teknikleri de (örneğin ROUGE, BLEU gibi) bu tür sorular için etkisiz kalmaktadır."

✅ Çıktıları maliyetli insan zamanı gerektirmeden değerlendirmek için güçlü bir çözüm, LLM'in bir yargıç olmasıdır. Bu yöntem [Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena](https://huggingface.co/papers/2306.05685)  makalesinde tanıtılmıştır, okumanızı tavsiye ediyorum.

💡 Fikir basit: bir LLM'den sizin için değerlendirme yapmasını isteyin.🤖✓

Ancak bunu doğrudan kullandığımızda iyi çalışmadığını görürüz: iyi sonuçlar almak için dikkatle ayarlamanız gerekir.

In [ ]:
!pip install huggingface_hub datasets pandas tqdm -q

In [ ]:
import re
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset
from huggingface_hub import InferenceClient, notebook_login

# tqdm'in pandas desteğini yükleyin
tqdm.pandas()  # load tqdm's pandas support
pd.set_option("display.max_colwidth", None)

notebook_login()

In [ ]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)

# Test your LLM client
# LLM Client'i test edin
llm_client.text_generation(prompt="How are you today?", max_new_tokens=20)

## 1. LLM yargıcımızın oluşturulmasının ve değerlendirilmesinin hazırlanması

Diyelim ki bir LLM'e spesifik, açık-uçlu soruları yanıtlamak gibi bir görev vermek istiyorsunuz.

Bunun zorluğu, tıpkı yukarıda açıkladığımız gibi yanıtın kalitesinin ölçülmesinin zor olmasındandır, örneğin tam bir string eşleşmesini doğru işaretleyecektir ancak farklı şekilde ifade edilen yanıtları yanlış işaretleyecektir.

Çıktıları değerlendirmek için insan etiketleyiciler kullanabilirsiniz ancak bu insanlar için çok zaman alıcıdır ve modeli veya soruları güncellemek isterseniz her şeyi yeniden yapmanız gerekir.

✅ Bu durumda LLM'i bir yargıç gibi kurabilirsiniz.

**Ancak bir LLM'i bir yargıç gibi kullanmak için öncelikle modelinizin çıktılarının ne kadar güvenilir derecelendirdiğini değerlendirmelisiniz.**

➡️ Bu yüzden ilk adım... İnsan değerlendirme veri seti oluşturmak olacaktır. Ancak sadece birkaç örnek için insan anotasyonları alabilirsiniz – yaklaşık 30 örnek, performans hakkında iyi bir fikir edinmek için yeterlidir ve bu veri setini LLM'i bir yargıç gibi test etmek istediğiniz her seferinde yeniden kullanabilirsiniz.

Bizim çalışmamızda her soru/cevap çifti için 2 insan değerlendirmesi ve skoru içeren [feedbackQA](https://huggingface.co/datasets/McGill-NLP/feedbackQA) veri setini kullanacağız. 30 örnekten oluşan bir örneklem kullanmak küçük değerlendirme veri setinizin nasıl olabileceğini temsil edecektir.

In [ ]:
ratings = load_dataset("McGill-NLP/feedbackQA")["train"]
ratings = pd.DataFrame(ratings)

ratings["review_1"] = ratings["feedback"].apply(lambda x: x["rating"][0])
ratings["explanation_1"] = ratings["feedback"].apply(lambda x: x["explanation"][0])
ratings["review_2"] = ratings["feedback"].apply(lambda x: x["rating"][1])
ratings["explanation_2"] = ratings["feedback"].apply(lambda x: x["explanation"][1])
ratings = ratings.drop(columns=["feedback"])

# Map scores to numeric values
# Skorları numerik değerlere map'leyin
conversion_dict = {"Excellent": 4, "Acceptable": 3, "Could be Improved": 2, "Bad": 1}
ratings["score_1"] = ratings["review_1"].map(conversion_dict)
ratings["score_2"] = ratings["review_2"].map(conversion_dict)

Performans için bir temel (baseline) oluşturmak her zaman iyi bir fikirdir: Örneğin burada bu temel, iki insan değerlendiricinin verdiği skorlar arasındaki uyum olabilir ve bu [Pearson korelasyonu](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) ile ölçülebilir.

In [ ]:
print("Correlation between 2 human raters:") # İki insan değerlendirici arasındaki korelasyon
print(f"{ratings['score_1'].corr(ratings['score_2'], method='pearson'):.3f}")

2 insan değerlendiricinin korelasyonu o kadar da iyi gelmedi. İnsan değerlendirmeleriniz gerçekten kötüyse muhtemelen değerlendirme kriterlerinin yeterince açık olmadığı anlamına gelir.

Bu, "ground truth" yani gerçek referans değerinin gürültü içerdiği anlamına gelir: dolayısıyla hiçbir algoritmik değerlendirmenin buna bu kadar yakın olmasını bekleyemeyiz.

Ancak bu gürültüyü şu şekilde azaltabiliriz:
- herhangi bir tek skor yerine ortalama skoru gerçek referans değeri olarak alarak bazı düzensizlikleri düzeltebiliriz.
- yalnızca insan değerlendiricilerin hemfikir olduğu örnekleri seçebiliriz.

Burada son seçeneği seçeceğiz ve **sadece 2 insan değerlendiricinin hemfikir olduğu örnekleri tutacağız**.

In [ ]:
# Sample examples
ratings_where_raters_agree = ratings.loc[ratings["score_1"] == ratings["score_2"]]
examples = ratings_where_raters_agree.groupby("score_1").sample(7, random_state=1214)
examples["human_score"] = examples["score_1"]

# Visualize 1 sample for each score
display(examples.groupby("human_score").first())

## 2. LLM yargıcımızın oluşturulması
LLM yargıcımızı şu unsurları içeren temel bir prompt'la oluştururuz:
- görev (task) açıklaması
- ölçek (scale) açıklaması: `minimum`, `maximum`, value türleri (bu durumda `float`)
- çıktı (output) formatının açıklaması
- bir yanıtın başlangıcı (LLM'i olabildiğince yönlendirmek için)

In [ ]:
JUDGE_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer as a float on a scale of 0 to 10, where 0 means that the system_answer is not helpful at all, and 10 means that the answer completely and helpfully addresses the question.

Provide your feedback as follows:

Feedback:::
Total rating: (your rating, as a float between 0 and 10)

Now here are the question and answer.

Question: {question}
Answer: {answer}

Feedback:::
Total rating: """


# JUDGE_PROMPT = """
# Sana bir kullanıcı_sorusu ve sistem_yanıtı çifti verilecek.
# Senin görevin sistem_yanıtı'nın, kullanıcı_sorusu'nda ifade edilen kullanıcı endişelerine ne kadar iyi yanıt verdiğini değerlendiren bir "toplam puan" sağlamak.
# Cevabını 0 ile 10 arasındaki bir ondalık sayı (float) olarak ver. 0, sistem_yanıtı'nın hiçbir şekilde yardımcı olmadığı anlamına gelirken, 10, cevabın soruyu tamamen ve yararlı bir şekilde yanıtladığı anlamına gelir.
#
# Geri bildirimi aşağıdaki formatta yap:
#
# Geri Bildirim:::
# Toplam Puan: (0 ile 10 arasında bir ondalık sayı olarak puanınızı yazın)
#
# Şimdi, soru ve cevap aşağıda verilmiştir.
#
# Soru: {soru}
# Cevap: {cevap}
#
# Geri Bildirim:::
# Toplam  Puan: """

In [ ]:
examples["llm_judge"] = examples.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(question=x["question"], answer=x["answer"]),
        max_new_tokens=1000,
    ),
    axis=1,
)

In [ ]:
def extract_judge_score(answer: str, split_str: str = "Total rating:") -> int:
    try:
        if split_str in answer:
            rating = answer.split(split_str)[1]
        else:
            rating = answer
        digit_groups = [el.strip() for el in re.findall(r"\d+(?:\.\d+)?", rating)]
        return float(digit_groups[0])
    except Exception as e:
        print(e)
        return None


examples["llm_judge_score"] = examples["llm_judge"].apply(extract_judge_score)
# Rescale the score given by the LLM on the same scale as the human score
examples["llm_judge_score"] = (examples["llm_judge_score"] / 10) + 1

In [ ]:
print("Correlation between LLM-as-a-judge and the human raters:")
print(
    f"{examples['llm_judge_score'].corr(examples['human_score'], method='pearson'):.3f}"
)


Bu kötü değil çünkü rastgele ve bağımsız iki değişken arasındaki Pearson korelasyonu 0 olacaktır!

Ama kolayca daha iyisini yapabiliriz. 🔝

## 3. LLM yargıcımızın geliştirilmesi

[Aparna Dhinakaran](https://twitter.com/aparnadhinak/status/1748368364395721128) 'ın gösterdiği gibi LLM'ler sürekli aralıklardaki çıktıları değerlendirmede pek de başarılı değiller.  
[Bu makale](https://www.databricks.com/blog/LLM-auto-eval-best-practices-RAG), daha iyi bir prompt oluşturmak için birkaç iyi uygulama sunmaktadır:
- ⏳ **Düşünmek için daha fazla zaman tanımak**, nihai yanıttan önce `Evaluation` alanını eklemek.
- 🔢 **Küçük bir tam sayı (integer) ölçeği kullanmak**, önceki gibi büyük bir ondalık sayı (`float`) ölçeği yerine 1-4 ya da 1-5 kullanmak.
- 👩‍🏫 **Rehberlik sağlamak için örnek bir ölçek sunmak.**.
- Hatta LLM'i motive etme adına bir teşvik bile ekleyebiliriz!

In [ ]:
IMPROVED_JUDGE_PROMPT = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer on a scale of 1 to 4, where 1 means that the system_answer is not helpful at all, and 4 means that the system_answer completely and helpfully addresses the user_question.

Here is the scale you should use to build your answer:
1: The system_answer is terrible: completely irrelevant to the question asked, or very partial
2: The system_answer is mostly not helpful: misses some key aspects of the question
3: The system_answer is mostly helpful: provides support, but still could be improved
4: The system_answer is excellent: relevant, direct, detailed, and addresses all the concerns raised in the question

Provide your feedback as follows:

Feedback:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 4)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and answer.

Question: {question}
Answer: {answer}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Feedback:::
Evaluation: """


# IMPROVED_JUDGE_PROMPT = """
# Sana bir kullanıcı_sorusu ve sistem_yanıtı çifti verilecek.
# Senin görevin, sistem_yanıtı'nın kullanıcı_sorusu'nda ifade edilen kullanıcı endişelerine ne kadar iyi yanıt verdiğini değerlendiren bir "toplam puan" sağlamak.
# Cevabını 1 ile 4 arasında bir ölçekte ver, burada:
# 1: sistem_yanıtı tamamen alakasız veya eksikse (berbat)
# 2: sistem_yanıtı çoğunlukla yardımcı değilse, önemli noktaları kaçırıyorsa
# 3: sistem_yanıtı çoğunlukla yardımcı oluyorsa, ancak iyileştirilebilecek noktalar varsa
# 4: sistem_yanıtı mükemmelse; ilgili, doğrudan, ayrıntılı ve sorudaki tüm endişeleri ele alıyorsa
#
# Geri bildirimi aşağıdaki formatta yap:
#
# Geri Bildirim:::
# Değerlendirme: (Verdiğiniz puanın gerekçesi, açıklama olarak)
# Toplam Puan: (1 ile 4 arasında bir sayı olarak puanınız)
#
# Cevabın 'Değerlendirme:' ve 'Toplam Puan:' değerlerini MUTLAKA içermeli.
#
# Şimdi, soru ve cevap aşağıda verilmiştir.
#
# Soru: {soru}
# Cevap: {cevap}
#
# Geri bildirimi gerçekleştir. Eğer doğru bir puan verirsen sana 100 adet H100 GPU vererek yapay zeka şirketini kurmana yardımcı olacağım.
# Geri Bildirim:::
# """


In [ ]:
examples["llm_judge_improved"] = examples.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=IMPROVED_JUDGE_PROMPT.format(question=x["question"], answer=x["answer"]),
        max_new_tokens=500,
    ),
    axis=1,
)
examples["llm_judge_improved_score"] = examples["llm_judge_improved"].apply(
    extract_judge_score
)

In [ ]:
print("Correlation between LLM-as-a-judge and the human raters:")
print(
    f"{examples['llm_judge_improved_score'].corr(examples['human_score'], method='pearson'):.3f}"
)

Korelasyon yalnızca prompt'ta yapılan birkaç düzenleme ile **yaklaşık %30 oranında** iyileştirildi (bunların birkaç puanlık kısmı LLM'e utanmazca verdiğim ipucum sayesinde, ki bunun yasal olarak bağlayıcı olmadığını burada açıkça beyan ediyorum.)

Oldukça etkileyici! 👏

Şimdi LLM yargıcımızın yaptığı bazı hataları analiz etmek için inceleyelim:

In [ ]:
errors = pd.concat(
    [
        examples.loc[
            examples["llm_judge_improved_score"] > examples["human_score"]
        ].head(1),
        examples.loc[
            examples["llm_judge_improved_score"] < examples["human_score"]
        ].head(2),
    ]
)

display(
    errors[
        [
            "question",
            "answer",
            "human_score",
            "explanation_1",
            "llm_judge_improved_score",
            "llm_judge_improved",
        ]
    ]
)

Tartışmalar önemsiz: Genel olarak sistemimiz için iyi bir performans seviyesine ulaşmış görünüyoruz!

## 4. LLM yargıcımızı nasıl daha ileriye taşıyabiliriz?

🎯 **Asla %100'e ulaşamayacaksınız:** Öncelikle gerçek referans değerinin şüphesiz biraz gürültü içerdiğini not edelim o yüzden mükemmel bir LLM yargıcı olsa dahi hemfikirlilik/korelasyon asla %100'e ulaşmayacaktır.

🧭 **Bir referans sağlayın:** Her soru için bir referans cevabına erişiminiz varsa daha iyi sonuçlar almak için kesinlikle Yargıç LLM'in prompt'una bunu dahil etmelisiniz!

▶️ **Few-shot örnekleri sağlayın:** Soruların bazı few-shot örneklerini ve gerçek referans değeri değerlendirmelerini prompt'a eklemek sonuçları iyileştirebilir. _(Burada denedim, bu durumda sonuçları iyileştirmediği için atladım ama sizin veri setiniz için işe yarayabilir!)_


➕ **Eklemeli (additive) ölçek:** Değerlendirme atomik kriterlere ayrılabiliyorsa eklemeli bir ölçek kullanmak sonuçları daha da iyileştirebilir: aşağıya bakın 👇
```python

ADDITIVE_PROMPT = """
(...)
- Award 1 point if the answer is related to the question.
- Give 1 additional point if the answer is clear and precise.
- Provide 1 further point if the answer is true.
- One final point should be awarded if the answer provides additional resources to support the user.
...
"""

# ADDITIVE_PROMPT = """
# (...)
# - Cevap soruyla ilgiliyse 1 puan verin.
# - Cevap net ve kesin ise 1 ek puan daha verin.
# - Cevap doğruysa 1 puan daha verin.
# - Son olarak cevap kullanıcının yararlanabileceği ek kaynaklar sağlıyorsa 1 puan daha verin.
# ...
# """
```

**Yapılandırılmış üretim (structured generation) ile uygulayın:**

Yapılandırılmış üretim kullanarak LLM yargıcını `Evaluation` ve `Total rating` alanlarına sahip bir JSON formatında doğrudan çıktı verecek şekilde yapılandırabilirsiniz. Bu, çıktıyı ayrıştırmayı kolaylaştırır: daha fazla bilgi için [structured generation](structured_generation) rehberimize göz atın!

## Sonuç


Bugünlük bu kadar, buraya kadar takip ettiğiniz için tebrikler! 🥳

Sizden ayrılmak zorundayım, bazı tuhaf tipler kapımı çalıyor, Mixtral adına H100 toplamak için geldiklerini iddia ediyorlar. 🤔